In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Read data

In [113]:
#wide_info_auto_trans = pd.read_csv('/home/mam_jupyter/jupyter_dir/artefact/leads_scoring_model/wide_info_auto.csv', delimiter = ',', index_col = 0)

In [131]:
lm_auto = pd.read_csv('lift_matrix_auto.csv')

In [132]:
lm_auto.head()

,action,0,1,Total action population,action_origin,right,comment
0,1.0,0.999359,1.004554,156184,d_fir_leads_time_weekday,NaN,NaN
1,0.0,1.016487,0.882824,6070,d_fir_leads_time_weekday,NaN,NaN
2,1.0,1.000000,1.000000,162254,d_fir_card_time_weekday,NaN,NaN
3,0.0,1.052333,0.628070,117784,d_fir_trail_weekday,NaN,NaN
4,1.0,0.861391,1.985100,44470,d_fir_trail_weekday,NaN,NaN


In [133]:
len(lm_auto['action_origin'].unique())

139

### Filter actions with small lift score

In [134]:
lm_auto['failed_score'] = [1 if i <= 1.1 and i >= 0.9 else 0 for i in lm_auto['0']]
lm_auto['deal_score'] = [1 if i <= 1.1 and i >= 0.9 else 0 for i in lm_auto['1']]

In [135]:
lm_auto.head()

,action,0,1,Total action population,action_origin,right,comment,failed_score,deal_score
0,1.0,0.999359,1.004554,156184,d_fir_leads_time_weekday,NaN,NaN,1,1
1,0.0,1.016487,0.882824,6070,d_fir_leads_time_weekday,NaN,NaN,1,0
2,1.0,1.000000,1.000000,162254,d_fir_card_time_weekday,NaN,NaN,1,1
3,0.0,1.052333,0.628070,117784,d_fir_trail_weekday,NaN,NaN,1,0
4,1.0,0.861391,1.985100,44470,d_fir_trail_weekday,NaN,NaN,0,0


In [136]:
def count_distribution(df, col1, col2):
    '''
    Calculate the distribution group by given columns given:
    df(dataframe): input dataframe;
    col1(string): column used to group by
    col2(string): column used to group by    
    '''
    dis_df = df[['Total action population',col1, col2]].groupby([col1, col2], as_index = False).sum().\
    merge(df[['Total action population',col1]].groupby([col1], as_index = False).sum(), how = 'left', on = col1)    
    dis_df['pop_rate'] = dis_df['Total action population_x'] / dis_df['Total action population_y']
    return dis_df

In [137]:
lm_auto = lm_auto.merge(count_distribution(lm_auto, 'action_origin', 'action')[['action', 'pop_rate','action_origin']], on = ['action','action_origin'], how = 'left')

In [138]:
no_difference_list = []
def no_difference_2(feature_list):    
    for feature in feature_list:
        feature_action_list = list(lm_auto[lm_auto['action_origin'] == feature]['action'])
        for action in feature_action_list:
            df = lm_auto[(lm_auto['action'] == action)&(lm_auto['action_origin']==feature)]
            if ((df['failed_score'] == 1) & (df['deal_score'] == 1) & (df['pop_rate'] >= 0.90)).bool() == True:
                value = feature
            else:
                value = np.nan
            no_difference_list.append(value)
    return no_difference_list

In [139]:
no_difference_feature = list(set(no_difference_2(list(lm_auto['action_origin'].unique()))))

In [140]:
lm_auto = lm_auto[~lm_auto['action_origin'].isin(no_difference_feature)]

In [141]:
len(lm_auto['action_origin'].unique())

86

In [142]:
lm_auto.to_csv('lm_remained_auto.csv',index = False)

In [143]:
lm_auto[['action_origin', 'comment']].drop_duplicates().to_csv('remained_auto.csv', index = False)